In [86]:
from bs4 import BeautifulSoup 
import requests as r
import pandas as pd
import re
from fake_useragent import UserAgent

In [ ]:
# Vs 2
# of finding ways to scrape used cars 
# built using bs4 and requests as opposed to scrapy in order to keep it low-level & simplistic w/o having to rely on virtual envs
# enabling code to run 'in-place'; no need to have other code where spiders are set up etc 

In [117]:
url='https://www.sgcarmart.com/used_cars/listing.php?BRSR=0&RPG=100&AVL=2&VEH=0'
# url lists 1st page; displaying 100 used-cars 
request=r.get(url)
soup=BeautifulSoup(request.text,'html.parser')

max_price=50000 # displays the max price 1 is willing to spend on a used car; in this case 50K

start_urls=[]
for e in soup.find('div',class_='price_child dropdown_child').find_all('a')[1:]: 
    # goes to the website code and searches for the relevant links which display used cars with prices below the max_price 
    # as the prices are segmented (10-15k, 20-25k etc, necessary to visit each url) 
    max_range=int(re.findall('\d+,?\d+',str(e.text))[-1].replace(',',''))
    
    if max_price>=max_range: # adds these urls to the start_urls list if the max price (price at the end; e.g. 10-15k; looks at the 15k), is 
        # less than/equal to the max price
        url='https://www.sgcarmart.com/used_cars/'+(e.get('href'))
        start_urls.append(url)
print(start_urls)
title=(f'Cars_below_{max_price/1000:.0f}k') # title for excel sheet (referenced at end)

['https://www.sgcarmart.com/used_cars/listing.php?PRC=10&RPG=100&VEH=0&AVL=2', 'https://www.sgcarmart.com/used_cars/listing.php?PRC=9&RPG=100&VEH=0&AVL=2']


In [113]:
ua=UserAgent() # initialises a user agent


In [118]:

overall=pd.DataFrame(columns=['Car_name','Price','Car_type','Reg_date','COE_term_len','No_owners','Car_url','Description'])
# dataframe created
for url in start_urls:  # to visit each url in the start_urls list 
    while True:

        agent=str(ua.random) # creates a random user agent (necessary to prevent multiple service requests to the server from the same user agent)
        # which in turn can flag the computer 
        # also the reason why this code cannot be run frequently
        while 'Windows' not in agent: # checks if agent has Windows keyword (being run from a windows computer); ideal that website used is 
            # templated for windows as well
            agent=str(ua.random)
        header={'User-Agent':agent} 

        try:
            request=r.get(url,headers=header) 
            soup=BeautifulSoup(request.text,'html.parser')
        except: # in the event of failure to get the request; code is exited
            print('Code Exited')
            break 
        else:
            pass
        for e in soup.find_all('table',{'style':'table-layout: fixed;'}): # extracts necessary info
            if (e.find('strong').find('a').get('href')).startswith('info.php'):
                name=((e.find('strong').find('a').text))
                car_url=('https://www.sgcarmart.com/used_cars/'+(e.find('strong').find('a').get('href')))
                c_request=r.get(car_url,headers=header) # to visit each car pages url (for extra info not available on the main page)
                c_soup=BeautifulSoup(c_request.text,'html.parser')
                
                reg_date=c_soup.find_all('tr',class_='row_bg')[-1].find_all('td')[-1].text.strip().split('(')[0]
                coe_term=re.findall(r'\((.*) C.*\)',c_soup.find_all('tr',class_='row_bg')[-1].find_all('td')[-1].text.strip())
                if len(coe_term)>0:
                    coe_term_len=coe_term[0]
                else:
                    coe_term_len=''  
                for x in c_soup.find_all('strong'):
                    if x.text=='No. of Owners':
                        no_owners=(x.parent.find_next_sibling().text.strip())
            else:
                continue 
            price=(e.find('td',{'class':'font_red'}).text.strip()[1:].replace(',',''))
            car_type=(e.find('a',class_="link_black nounderline").text)
            try:
                description=(e.find('div',class_='two_line_text').text)
            except:
                description='' 
            # info is collated into a row and added to a new row of the overall dataframe
            new_row={'Car_name':name,'Price':price,'Car_type':car_type, 'Reg_date':reg_date,
                     'COE_term_len':coe_term_len,'No_owners':no_owners,'Car_url':car_url,'Description':description}
            overall.loc[len(overall)]=new_row

        if soup.find('a',class_='pagebar')!=None and soup.find_all('a',class_='pagebar')[-1].text.strip()[0].upper()=='N':
            url='https://www.sgcarmart.com'+(soup.find_all('a',class_='pagebar')[-1].get('href'))
            request=r.get(url,headers=header) # request !!!!
            soup=BeautifulSoup(request.text,'html.parser')
        else:
            break


In [119]:
overall['Price']=overall['Price'].astype('int32') # saves price information as numerical data (done separately from the main code 
# in event that str data is captured which could throw an error in the main block of code)


,Car_name,Price,Car_type,Reg_date,COE_term_len,No_owners,Car_url,Description
0,Nissan Sylphy 1.6A,47800,Sedan,16-Apr-2016,3yrs 23days,3,https://www.sgcarmart.com/used_cars/info.php?I...,Consignment unit! Good for mid size family wit...
1,Mitsubishi Fuso Canter FEB21,48800,Truck,13-Nov-2015,2yrs 7mths 20days,1,https://www.sgcarmart.com/used_cars/info.php?I...,
2,Kia Cerato K3 1.6A SX Sunroof,45800,Sedan,12-Nov-2015,2yrs 7mths 19days,3,https://www.sgcarmart.com/used_cars/info.php?I...,10 years C&C engine warranty. 100% flexi loan ...
3,Nissan Qashqai 1.2A DIG-T Premium,45300,SUV,20-Feb-2016,2yrs 10mths 27days,2,https://www.sgcarmart.com/used_cars/info.php?I...,Only one careful owner view to appreciate no r...
4,Audi A3 Sportback 1.4A TFSI S-tronic Ambiente,48800,Hatchback,28-Dec-2015,2yrs 9mths 4days,2,https://www.sgcarmart.com/used_cars/info.php?I...,"Good condition, workshop/STA evaluation availa..."
...,...,...,...,...,...,...,...,...
1222,Toyota Vellfire 2.4A Z G-Edition Moonroof,55000,MPV,31-Mar-2014,1yr 7days,4,https://www.sgcarmart.com/used_cars/info.php?I...,Well maintained. Carplate will be retained. 6 ...
1223,Toyota Dyna 150 3.0M,50800,Truck,21-Mar-2016,2yrs 11mths 25days,2,https://www.sgcarmart.com/used_cars/info.php?I...,well kept. Good vehicle. Easy to maintain. 100...
1224,Audi A4 1.8A TFSI MU,56800,Luxury,27-Aug-2015,2yrs 5mths 3days,1,https://www.sgcarmart.com/used_cars/info.php?I...,2015 Audi A4. 1 owner only. low mileage done. ...
1225,BMW 2 Series 216d Gran Tourer Luxury,57555,MPV,19-Nov-2015,2yrs 7mths 26days,1,https://www.sgcarmart.com/used_cars/info.php?I...,"Cheapest on Market, 100% Loan! 1 owner unit on..."


In [120]:
overall['Reg_year']=overall['Reg_date'].apply(lambda x:x.split('-')[-1])
# done to get just the year of registration of the car (which generally tallies with the manufacturing of the car (closer year>recently made))

In [121]:
def coe_len(term): # function to determine the coe length (yrs mths and days) into days only (assuming 1 mth has 30 days & 1 year has 365 days)
    if 'yr' in term and 'mth' in term and 'day' in term:
        years=int(re.findall(r'(\d+)yr',term)[0])
        mths=int(re.findall(r'(\d+)mth',term)[0])
        days=int(re.findall(r'(\d+)day',term)[0])
    elif 'yr' in term and 'mth' in term:
        years=int(re.findall(r'(\d+)yr',term)[0])
        mths=int(re.findall(r'(\d+)mth',term)[0])
        days=0
    elif 'yr' in term and 'day' in term:
        years=int(re.findall(r'(\d+)yr',term)[0])
        days=int(re.findall(r'(\d+)day',term)[0])
        mths=0
    elif 'mth' in term and 'day' in term:
        mths=int(re.findall(r'(\d+)mth',term)[0])
        days=int(re.findall(r'(\d+)day',term)[0])
        years=0
    elif 'yr' in term:
        years=int(re.findall(r'(\d+)yr',term)[0])
        mths=0 
        days=0 
    elif 'mth' in term:
        mths=int(re.findall(r'(\d+)mth',term)[0])
        years=0 
        days=0 
    else:
        try:
            days=int(re.findall(r'(\d+)day',term)[0])
            years=0 
            mths=0  
        except:
            days=0 
            years=0 
            mths=0
    return (years*365+mths*30+days) 

overall['COE_term_days']=overall['COE_term_len'].apply(coe_len)

In [122]:
overall['dollar_p_day']=overall['Price']/overall['COE_term_days'] # observes how 'worth' the car is; lower value ideal as for same amount; 
# can possess the car for longer
overall

,Car_name,Price,Car_type,Reg_date,COE_term_len,No_owners,Car_url,Description,Reg_year,COE_term_days,dollar_p_day
0,Nissan Sylphy 1.6A,47800,Sedan,16-Apr-2016,3yrs 23days,3,https://www.sgcarmart.com/used_cars/info.php?I...,Consignment unit! Good for mid size family wit...,2016,1118,42.754919
1,Mitsubishi Fuso Canter FEB21,48800,Truck,13-Nov-2015,2yrs 7mths 20days,1,https://www.sgcarmart.com/used_cars/info.php?I...,,2015,960,50.833333
2,Kia Cerato K3 1.6A SX Sunroof,45800,Sedan,12-Nov-2015,2yrs 7mths 19days,3,https://www.sgcarmart.com/used_cars/info.php?I...,10 years C&C engine warranty. 100% flexi loan ...,2015,959,47.758081
3,Nissan Qashqai 1.2A DIG-T Premium,45300,SUV,20-Feb-2016,2yrs 10mths 27days,2,https://www.sgcarmart.com/used_cars/info.php?I...,Only one careful owner view to appreciate no r...,2016,1057,42.857143
4,Audi A3 Sportback 1.4A TFSI S-tronic Ambiente,48800,Hatchback,28-Dec-2015,2yrs 9mths 4days,2,https://www.sgcarmart.com/used_cars/info.php?I...,"Good condition, workshop/STA evaluation availa...",2015,1004,48.605578
...,...,...,...,...,...,...,...,...,...,...,...
1222,Toyota Vellfire 2.4A Z G-Edition Moonroof,55000,MPV,31-Mar-2014,1yr 7days,4,https://www.sgcarmart.com/used_cars/info.php?I...,Well maintained. Carplate will be retained. 6 ...,2014,372,147.849462
1223,Toyota Dyna 150 3.0M,50800,Truck,21-Mar-2016,2yrs 11mths 25days,2,https://www.sgcarmart.com/used_cars/info.php?I...,well kept. Good vehicle. Easy to maintain. 100...,2016,1085,46.820276
1224,Audi A4 1.8A TFSI MU,56800,Luxury,27-Aug-2015,2yrs 5mths 3days,1,https://www.sgcarmart.com/used_cars/info.php?I...,2015 Audi A4. 1 owner only. low mileage done. ...,2015,883,64.326161
1225,BMW 2 Series 216d Gran Tourer Luxury,57555,MPV,19-Nov-2015,2yrs 7mths 26days,1,https://www.sgcarmart.com/used_cars/info.php?I...,"Cheapest on Market, 100% Loan! 1 owner unit on...",2015,966,59.580745


In [123]:
overall.to_excel(f'{title}.xlsx',index=False) # sends df information to an excel sheet in the same host folder; no index present